In [182]:
from __future__ import division

import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats
import matplotlib.pylab as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import statsmodels.sandbox.stats as smsb

from rpy2.robjects import r
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


## Original R material from from David Howell's site: https://www.uvm.edu/~dhowell/StatPages/Permutation%20Anova/PermTestsAnova.html

## (and Manly, B. F. J. (2007) Randomization, Bootstrap, and Monte Carlo Methods in Biology (3rd ed.), London: Chapman & Hall. - p 144 before that...)

In [160]:
d_dict = {}
d_dict['ants'] = np.array([13, 242, 105, 182, 21, 7, 8, 59, 20, 24, 312, 68, 515, 488, 88, 460, 1223, 990, 18, 44, 21, 140, 40, 27])
d_dict['size'] = np.tile(np.array([1,1,1,2,2,2]), reps = 4)
d_dict['months'] = np.repeat(np.array([1,2,3,4]), 6)

print(d_dict['ants'])
print(d_dict['size'])
print(d_dict['months'])

df = pd.DataFrame(d_dict)

df['size'] = pd.Categorical(df['size'])
df['months'] = pd.Categorical(df['months'])
                                
mod = smf.ols('ants ~ size + months + size:months', data = df).fit()
an_mod = sm.stats.anova_lm(mod, typ = 2)
print(an_mod)

[  13  242  105  182   21    7    8   59   20   24  312   68  515  488   88
  460 1223  990   18   44   21  140   40   27]
[1 1 1 2 2 2 1 1 1 2 2 2 1 1 1 2 2 2 1 1 1 2 2 2]
[1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 4]
                   sum_sq    df          F    PR(>F)
size         1.461720e+05   1.0   4.469905  0.050548
months       1.379495e+06   3.0  14.061540  0.000095
size:months  2.940095e+05   3.0   2.996912  0.061712
Residual     5.232220e+05  16.0        NaN       NaN


In [161]:
%R ants <- c(13, 242, 105, 182, 21, 7, 8, 59, 20, 24, 312, 68, 515, 488, 88, 460, 1223, 990, 18, 44, 21, 140, 40, 27)
%R size <- as.factor(rep(1:2, each = 3, times = 4)) 
%R months <- as.factor(rep(1:4, each = 6))
%R print(ants)
%R print(size)
%R print(months)

%R mod1 <- lm(ants ~ size + months + size:months)
%R print(summary(aov(mod1)))

 [1]   13  242  105  182   21    7    8   59   20   24  312   68  515  488   88
[16]  460 1223  990   18   44   21  140   40   27


 [1] 1 1 1 2 2 2 1 1 1 2 2 2 1 1 1 2 2 2 1 1 1 2 2 2
Levels: 1 2


 [1] 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 4
Levels: 1 2 3 4


            Df  Sum Sq Mean Sq F value   Pr(>F)    
size         1  146172  146172   4.470   0.0505 .  
months       3 1379495  459832  14.062 9.49e-05 ***
size:months  3  294009   98003   2.997   0.0617 .  
Residuals   16  523222   32701                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [162]:
def anova_perm(formula, shuffle_var, dataframe, n_shuffles):
    '''
    Uses Manly's method of unrestrained reshuffling across levels to calculate null distribution & hypothesis tests
    % shuffles > obs
    
    Input:
    formula - Patsy compliant formula for the model
    dataframe - Pandas dataframe in long form with the data to be modelled.
    n_shuffles - number of reshuffles to use
    seed - Seed for reproducability
    
    Output:
    obs_f - Observed statistic from the f test
    perm_f - Permutated statistic from the f test
    p - % shuffles > obs 
    '''

    t = np.round(n_shuffles / 4, decimals = 0)
    
    #Get data info
    r, c = dataframe.shape
    
    # Calculate observed linear mixed model
    import warnings

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        obs_model = model = smf.ols(formula, data = dataframe).fit(reml = False)
    obs_model_f = sm.stats.anova_lm(obs_model, typ = 2)
    obs_f = obs_model_f[obs_model_f.index != 'Residual']['F']
    
    #Capture information from output wald results (minus intercept)
    terms = obs_f.index.values
    
    #Preallocate memory
    perm_f = {term: np.zeros(n_shuffles) for term in terms}    
    
    #Loop selecting random indicies, running ANOVA and collecting test statistic
    for n in range(0, n_shuffles):
        if n % t == 0:
            print((n / n_shuffles) * 100)
            
        shuffle_vals = np.random.choice(dataframe[shuffle_var], size = len(dataframe), replace = False)
        df_shuffle = dataframe.copy()
        df_shuffle[shuffle_var] = shuffle_vals
        
        perm_model = smf.ols(formula, data = df_shuffle).fit(reml = False)
        perm_model_f = sm.stats.anova_lm(perm_model, typ = 2)
        perm_f_shuffle = perm_model_f[perm_model_f.index != 'Residual']['F']
        
        for term in terms:
            perm_f[term][n] = perm_f_shuffle.loc[term]

    
    p = {term: sum(perm_f[term] >= obs_f[term]) / n_shuffles for term in terms}
    print('Complete')
    return(obs_f, perm_f, p)

In [184]:
dataframe = df
n_shuffles = 100
formula = 'ants ~ size + months + size:months'
shuffle_var = 'ants'

obs, perm, p = anova_perm('ants ~ size + months + size:months', shuffle_var, df, 5000)

0.0
25.0
50.0
75.0


In [185]:
p

{'months': 0.0,
 'size': 0.040000000000000001,
 'size:months': 0.055199999999999999}

## Verify in R w/ original code

In [187]:
%R mod1 <- lm(ants ~ size + months + size:months)
%R ANOVA <- summary(aov(mod1))
%R cat( " The standard ANOVA for these data follows ","\n")
%R Fsize <-  ANOVA[[1]]$"F value"[1]   # Saving F values for future use
%R Fmonths <-  ANOVA[[1]]$"F value"[2]
%R Finteract <-  ANOVA[[1]]$"F value"[3]
%R print(summary(aov(mod1)))

%R print( "Resampling as in Manly with unrestricted sampling of observations. ")
%R # Now start resampling
%R nreps <- 5000 
%R FS <- numeric(nreps)    #Set up space to store F values as calculated.
%R FM <- numeric(nreps)  
%R FSM <- numeric(nreps)
%R FS[1] <- Fsize          # The first F of our 5000 
%R FM[1] <- Fmonths
%R FSM[1] <- Finteract

r('''
for (i in 1:nreps) {
  newants <- sample(ants, 24)
  mod2 <- lm(newants ~ size + months + size:months)
  b <- summary(aov(mod2))
  FS[i] <- b[[1]]$"F value"[1]
  FM[i] <- b[[1]]$"F value"[2]
  FSM[i] <- b[[1]]$"F value"[3]
  }''')
%R print(i)
%R probS <- length(FS[FS >= Fsize + .Machine$double.eps ^0.5])/nreps
%R probM <- length(FM[FM >= Fmonths+ .Machine$double.eps ^0.5])/nreps       
%R probSM  <-  length(FSM[FSM >= Finteract + .Machine$double.eps ^0.5])/nreps
### The addition of "+ .Machine$double.eps" is an aid against two numbers that differ only by
### floating point computer calculations at the extreme.

%R cat(" The probability value for the interaction is ",probSM, "\n")
%R cat(" The probability value for Size is ", probS, "\n")
%R cat(" The probability value for Months is ", probM, "\n")

 The standard ANOVA for these data follows  


            Df  Sum Sq Mean Sq F value   Pr(>F)    
size         1  146172  146172   4.470   0.0505 .  
months       3 1379495  459832  14.062 9.49e-05 ***
size:months  3  294009   98003   2.997   0.0617 .  
Residuals   16  523222   32701                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


[1] "Resampling as in Manly with unrestricted sampling of observations. "


[1] 5000


 The probability value for the interaction is  0.051  


 The probability value for Size is  0.043  


 The probability value for Months is  2e-04  


In [183]:
p

{'months': 0.00040000000000000002,
 'size': 0.047,
 'size:months': 0.053600000000000002}

### Close enough for 5,000 perms.